In [ ]:
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
!unzip /content/drive/MyDrive/10000.zip

Streaming output truncated to the last 5000 lines.
  inflating: 10000/7/7_427.jpg       
  inflating: 10000/7/7_527.jpg       
  inflating: 10000/7/7_104.jpg       
  inflating: 10000/7/7_543.jpg       
  inflating: 10000/7/7_805.jpg       
  inflating: 10000/7/7_698.jpg       
  inflating: 10000/7/7_228.jpg       
  inflating: 10000/7/7_169.jpg       
  inflating: 10000/7/7_493.jpg       
  inflating: 10000/7/7_121.jpg       
  inflating: 10000/7/7_122.jpg       
  inflating: 10000/7/7_198.jpg       
  inflating: 10000/7/7_846.jpg       
  inflating: 10000/7/7_926.jpg       
  inflating: 10000/7/7_579.jpg       
  inflating: 10000/7/7_439.jpg       
  inflating: 10000/7/7_937.jpg       
  inflating: 10000/7/7_423.jpg       
  inflating: 10000/7/7_540.jpg       
  inflating: 10000/7/7_302.jpg       
  inflating: 10000/7/7_343.jpg       
  inflating: 10000/7/7_454.jpg       
  inflating: 10000/7/7_240.jpg       
  inflating: 10000/7/7_580.jpg       
  inflating: 10000/7/7_25.jpg        

In [282]:
import tensorflow as tf
import keras, os, cv2
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
import keras_tuner as kt
import joblib
import numpy as np
from sklearn.utils import shuffle
import seaborn as sns; sns.set(font_scale=1.4)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,  f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPClassifier
import pandas as pd

In [283]:
def data_Loading(ImagePath):
  #append folders in list
  Folders = os.listdir(ImagePath)

  #append image names
  titles = []
  for Folder in Folders:
    titles.append(os.listdir(os.path.join(ImagePath,Folder)))

  #convert names from list to numpy array for ease of use
  titles = np.array(titles)
  print("Image shape is : ",titles.shape)
  return Folders, titles

In [284]:
def create_lables(titles):
  row ,col = titles.shape
  lables = []

  for i in range(row):
      for j in range(col):
          lables.append(int(titles[i][j][0]))

  #convert lables from list to numpy array for ease of use
  lables = np.array(lables)

  print("lables shape is : ",lables.shape)
  print("lables : ",lables[:10])
  return lables

In [285]:
def image_preprocessing(ImagePath,titles,Folders):
  images = []
  img_size =(28,28)
  for Folder in Folders:
    for image in titles[Folders.index(Folder)]:
      #read image path
      path = os.path.join(ImagePath,Folder,image)
      #conevrt from rgb to grayscale for speed the training process
      img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
      #all images should be in one size ---> 28 * 28
      img = cv2.resize(img,img_size)
      # image reshape to be suitable for convolution process
      img = img.reshape(28,28,1)
      images.append(img)

  images = np.array(images)
  print("images shape is : ",images.shape)
  print("an image shape is : ",images[0].shape)
  return images

In [286]:
#rearrange data for better trainning
def data_shuffling(images,lables):
  images,lables = shuffle(images,lables,random_state=42)
  return images,lables

In [287]:
#reshape data to be suitable for others model
def data_reshaping(x):
  x = x.reshape(x.shape[0],784)
  print("train data shape : ",x.shape)
  return x

In [288]:
#pixels normalization for faster training
def images_normalization(images):
  images = images/255.0
  return images

In [289]:
#data splitting 80% trainning 20 test
def data_splitting(images,lables):
  xtrain, xtest, ytrain, ytest=train_test_split(images, lables,test_size=0.2,random_state=42,shuffle=True)
  print("xtrain shape : ",xtrain.shape)
  print("xtes shape : ",xtest.shape)
  print("ytrain shape : ",ytrain.shape)
  print("ytest shape : ",ytest.shape)
  return xtrain, xtest, ytrain, ytest

In [290]:
  ImagePath = "/content/10000"
  print("-------------image preprocessing------------")
  folders, titles = data_Loading(ImagePath)
  lables = create_lables(titles)
  images = image_preprocessing(ImagePath,titles,folders)
  images,lables = data_shuffling(images,lables)
  images = images_normalization(images)
  xtrain, xtest, ytrain, ytest = data_splitting(images,lables)

-------------image preprocessing------------
Image shape is :  (10, 1000)
lables shape is :  (10000,)
lables :  [2 2 2 2 2 2 2 2 2 2]
images shape is :  (10000, 28, 28, 1)
an image shape is :  (28, 28, 1)
xtrain shape :  (8000, 28, 28, 1)
xtes shape :  (2000, 28, 28, 1)
ytrain shape :  (8000,)
ytest shape :  (2000,)


In [291]:
# Neural network model with CNN and keras tuner for faster and better trainning
def nn_mmodel(xtrain, ytrain, xtest, ytest):
  def build_nnmodel(hp):
      model = keras.models.Sequential()
      model.add(keras.layers.Conv2D(
          filters=hp.Int('filters', min_value=32, max_value=128, step=32),
          kernel_size=hp.Choice('kernel_size', values=[3, 5]),
          activation='relu',
          input_shape=(28, 28, 1)
      ))
      #maxpooling for matrix with less size and faster computions,extract important edges
      model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
      #normalizing after each min-batch for faster training
      model.add(keras.layers.BatchNormalization())
      #flatten for convert convoluation matrix to vector to suitable neural netwrok
      model.add(keras.layers.Flatten())
      #dropout for cutting some random connections between neuron
      model.add(keras.layers.Dropout(0.3))

      model.add(keras.layers.Dense(8, activation='relu'))
      model.add(keras.layers.Dense(128, activation='relu'))
      model.add(keras.layers.Dense(4, activation='relu'))
      model.add(keras.layers.Dense(10, activation='softmax'))

      optimizer = keras.optimizers.Adam(
          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
      )
      model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
      return model

  # a tuner for find best value for hyperparameter
  tuner = kt.Hyperband(
      build_nnmodel,
      objective='val_accuracy',
      max_epochs=10,
      factor=3,
      directory='my_dir',
      project_name='digit_class'
  )

  # Search for best hyperparameters
  tuner.search(xtrain, ytrain, epochs=10, validation_data=(xtest, ytest))

  # get the best hyperparameters and model
  best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
  nn_model = tuner.hypermodel.build(best_hps)

  # model training
  nn_model.fit(xtrain, ytrain, epochs=50,validation_data=(xtest, ytest))
  return nn_model

In [292]:
def model_evaluation(model,xtest,ytest):
  xtest = data_reshaping(xtest)
  y_pred = model.predict(xtest)
  accuracy = accuracy_score(ytest, y_pred)
  print("predection classes", y_pred[:10])
  print("true classes", ytest[:10])
  print("Accuracy: {:.2f}%".format(accuracy * 100))
  class_report = classification_report(ytest, y_pred)
  f1score = f1_score(ytest, y_pred, average='weighted')
  precisionscore = precision_score(ytest, y_pred, average='weighted')
  recallscore = recall_score(ytest, y_pred, average='weighted')
  print(" Accuracy: {:.2f}%".format(accuracy * 100))
  print(" f1_score: {:.2f}%".format(f1score * 100))
  print(" precision_score: {:.2f}%".format(precisionscore * 100))
  print(" recall_score: {:.2f}%".format(recallscore * 100))
  # print("classification_report: \n",class_report)
  conf_matrix = confusion_matrix(ytest, y_pred)
  print("confusion_matrix: \n",confusion_matrix(ytest, y_pred))
  return accuracy,class_report,conf_matrix

In [293]:
def nn_evaluation(nn_model, xtest,ytest):
  y_pred = nn_model.predict(xtest)
  y_pred = np.argmax(y_pred, axis=1)
  accuracy = accuracy_score(ytest, y_pred)
  print("predection classes", y_pred[:10])
  print("true classes", ytest[:10])
  class_report = classification_report(ytest, y_pred)
  conf_matrix = confusion_matrix(ytest, y_pred)
  f1score = f1_score(ytest, y_pred, average='weighted')
  precisionscore = precision_score(ytest, y_pred, average='weighted')
  recallscore = recall_score(ytest, y_pred, average='weighted')
  print("Neural Network Accuracy: {:.2f}%".format(accuracy * 100))
  print("Neural Network f1_score: {:.2f}%".format(f1score * 100))
  print("Neural Network precision_score: {:.2f}%".format(precisionscore * 100))
  print("Neural Network recall_score: {:.2f}%".format(recallscore * 100))

  # print("Neural Network classification_report: \n",class_report)
  print("Neural Network confusion_matrix: \n",conf_matrix)
  return accuracy,class_report,conf_matrix

In [294]:
print("-------------cnn_model------------")
nn_model = nn_mmodel(xtrain, ytrain, xtest, ytest)


-------------cnn_model------------
Reloading Tuner from my_dir/digit_class/tuner0.json
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.1958 - loss: 2.1143 - val_accuracy: 0.4850 - val_loss: 1.9911
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.4939 - loss: 1.3343 - val_accuracy: 0.5405 - val_loss: 1.2633
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 42ms/step - accuracy: 0.7167 - loss: 0.8131 - val_accuracy: 0.7610 - val_loss: 0.7093
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.7903 - loss: 0.6077 - val_accuracy: 0.6975 - val_loss: 0.9283
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.8298 - loss: 0.4969 - val_accuracy: 0.6790 - val_loss: 1.2927
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.8680 - loss: 0.4159 - val_accuracy: 0.7835 - val_loss: 0.7638
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 44ms/step - accuracy: 0.8894 - loss: 0.3360 - val_accuracy: 0.7820 - val_loss: 0.7701
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.8807 - loss: 0.3666 - val_accuracy: 

In [295]:
nn_model.save("cnn_model.keras")

In [296]:
nn_model = tf.keras.models.load_model("cnn_model.keras")

In [297]:
nn_evaluation(nn_model, xtest,ytest)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
predection classes [5 0 8 9 6 7 7 2 7 9]
true classes [5 0 8 9 6 7 7 7 7 9]
Neural Network Accuracy: 87.60%
Neural Network f1_score: 87.70%
Neural Network precision_score: 88.63%
Neural Network recall_score: 87.60%
Neural Network confusion_matrix: 
 [[190   1   4   3   2   0   0   0   2   0]
 [  3 188   1   0  10   0   0   0   0   0]
 [  3   0 177   3   4   0   0   0   1   0]
 [  5   0  10 168   0   3   0   1   3   7]
 [  3   0   4   0 205   0   0   0   0   0]
 [  3   0   1  10   1 165   1   0   5  11]
 [ 18   1   1   2   1   1 183   0   4   0]
 [  0   2   9   0   3   0   2 165   1  32]
 [  9   0   3  16   1   4   0   0 167   5]
 [  6   0   1   7  11   0   0   2   1 144]]


(0.876,
 '              precision    recall  f1-score   support\n\n           0       0.79      0.94      0.86       202\n           1       0.98      0.93      0.95       202\n           2       0.84      0.94      0.89       188\n           3       0.80      0.85      0.83       197\n           4       0.86      0.97      0.91       212\n           5       0.95      0.84      0.89       197\n           6       0.98      0.87      0.92       211\n           7       0.98      0.77      0.86       214\n           8       0.91      0.81      0.86       205\n           9       0.72      0.84      0.78       172\n\n    accuracy                           0.88      2000\n   macro avg       0.88      0.88      0.88      2000\nweighted avg       0.89      0.88      0.88      2000\n',
 array([[190,   1,   4,   3,   2,   0,   0,   0,   2,   0],
        [  3, 188,   1,   0,  10,   0,   0,   0,   0,   0],
        [  3,   0, 177,   3,   4,   0,   0,   0,   1,   0],
        [  5,   0,  10, 168,   0,

In [300]:
def ova(linear_reg_model, X_train, y_train):
  X_train = data_reshaping(X_train)
  # Create a OneVsRestClassifier with LinearRegression as the base estimator
  ova_classifier = OneVsRestClassifier(linear_reg_model)
  # Train the classifier
  ova_classifier.fit(X_train, y_train)
  return ova_classifier




In [301]:
lr = LinearRegression()
ova_lr = ova(lr, xtrain, ytrain)


train data shape :  (8000, 784)


In [302]:
joblib.dump(ova_lr, 'ova_lr.pkl')

['ova_lr.pkl']

In [303]:
ova_lr = joblib.load('ova_lr.pkl')

In [304]:
accuracy,class_report,conf_matrix = model_evaluation(ova_lr,xtest,ytest)

train data shape :  (2000, 784)
predection classes [5 0 6 9 6 7 9 9 9 9]
true classes [5 0 8 9 6 7 7 7 7 9]
Accuracy: 67.40%
 Accuracy: 67.40%
 f1_score: 67.12%
 precision_score: 67.16%
 recall_score: 67.40%
confusion_matrix: 
 [[160   3   1   8   6   7   4   4   1   8]
 [  6 160   6   2   5   4   6   9   1   3]
 [  1   9 135   5   8  10   2   2   9   7]
 [  6   4  17 148   5   3   1   4   6   3]
 [  4  14   4   6 127   8  11   7  19  12]
 [ 10   4   1   8   6 120  22  11  10   5]
 [  4   6   8   2   5  14 170   1   1   0]
 [  1  16  10   6   4   9   4 124   6  34]
 [  7   8   3  14  21  12  12  16 103   9]
 [  8   3   8   8  11   4   2  22   5 101]]


In [305]:
def logistic_reg_grid_search(logistic_reg_model, xtrain, ytrain):
  # Define the hyperparameter grid to search
  xtrain = data_reshaping(xtrain)
  logistic_reg_param = {
    'C': [1],
    'penalty': ['l1','l2'],
    'max_iter': [1000],
    'verbose': [1],
    'solver': ['newton-cg', 'lbfgs']  # Add solver options
}

  # Create GridSearchCV object
  logistic_reg = GridSearchCV(logistic_reg_model, logistic_reg_param, cv=5, scoring='accuracy')  # cv=5 for 5-fold cross-validation

  # Fit the grid search to your data
  logistic_reg.fit(xtrain, ytrain)

  # Get the best hyperparameters and model
  best_model = logistic_reg.best_estimator_
  best_params = logistic_reg.best_params_
  print("Logistic Regression Best Hyperparameters:", best_params)
  print("Logistic Regression Best Model:", best_model)
  return logistic_reg

In [306]:
print("-------------logistic_model------------")
logistic_reg_model = LogisticRegression()
logistic_reg = logistic_reg_grid_search(logistic_reg_model, xtrain, ytrain)

-------------logistic_model------------
train data shape :  (8000, 784)
Newton-CG iter = 0
  Check Convergence
    max |gradient| <= tol: 0.02034332107843137 <= 0.0001 False
Newton-CG iter = 1
  Check Convergence
    max |gradient| <= tol: 0.03755689127219717 <= 0.0001 False
Newton-CG iter = 2
  Check Convergence
    max |gradient| <= tol: 0.005409087234430416 <= 0.0001 False
Newton-CG iter = 3
  Check Convergence
    max |gradient| <= tol: 0.004713771660088353 <= 0.0001 False
Newton-CG iter = 4
  Check Convergence
    max |gradient| <= tol: 0.00401743028565911 <= 0.0001 False
Newton-CG iter = 5
  Check Convergence
    max |gradient| <= tol: 0.00263436870068267 <= 0.0001 False
Newton-CG iter = 6
  Check Convergence
    max |gradient| <= tol: 0.0016919061577428706 <= 0.0001 False
Newton-CG iter = 7
  Check Convergence
    max |gradient| <= tol: 0.0010564071756761582 <= 0.0001 False
Newton-CG iter = 8
  Check Convergence
    max |gradient| <= tol: 0.0009439905761137239 <= 0.0001 False
Ne

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1193, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr

Newton-CG iter = 0
  Check Convergence
    max |gradient| <= tol: 0.020138627450980384 <= 0.0001 False
Newton-CG iter = 1
  Check Convergence
    max |gradient| <= tol: 0.038105395225417685 <= 0.0001 False
Newton-CG iter = 2
  Check Convergence
    max |gradient| <= tol: 0.005978232780365885 <= 0.0001 False
Newton-CG iter = 3
  Check Convergence
    max |gradient| <= tol: 0.0052184334314929505 <= 0.0001 False
Newton-CG iter = 4
  Check Convergence
    max |gradient| <= tol: 0.004171060188255378 <= 0.0001 False
Newton-CG iter = 5
  Check Convergence
    max |gradient| <= tol: 0.0025373560653862404 <= 0.0001 False
Newton-CG iter = 6
  Check Convergence
    max |gradient| <= tol: 0.0015036920837284383 <= 0.0001 False
Newton-CG iter = 7
  Check Convergence
    max |gradient| <= tol: 0.0009564231670263396 <= 0.0001 False
Newton-CG iter = 8
  Check Convergence
    max |gradient| <= tol: 0.0008144488591728804 <= 0.0001 False
Newton-CG iter = 9
  Check Convergence
    max |gradient| <= tol: 0.

In [307]:
joblib.dump(logistic_reg, 'logistic_reg.pkl')

['logistic_reg.pkl']

In [308]:
logistic_reg = joblib.load('logistic_reg.pkl')

In [309]:
accuracy,class_report,conf_matrix = model_evaluation(logistic_reg,xtest,ytest)

train data shape :  (2000, 784)
predection classes [5 0 8 9 6 7 9 9 9 9]
true classes [5 0 8 9 6 7 7 7 7 9]
Accuracy: 75.85%
 Accuracy: 75.85%
 f1_score: 75.72%
 precision_score: 75.76%
 recall_score: 75.85%
confusion_matrix: 
 [[167   3   1   4   3   7   2   4   4   7]
 [  5 175   5   2   1   2   0  11   1   0]
 [  3   4 154   5   3   6   2   1   7   3]
 [  6   3  11 147   3   2   1   5  10   9]
 [  3  13   5   3 154   4   7   7   7   9]
 [  8   0   1   6   3 145  20   4   7   3]
 [  2   1   3   0   3  17 178   1   6   0]
 [  2   6  13   7   7   1   2 155   3  18]
 [  3   5   4   9  19  16   7  12 124   6]
 [  8   0   7   7   8   2   0  19   3 118]]


In [310]:
def multinominalNB_param_grid_search(naive_bayes_model, xtrain, ytrain):
  xtrain = data_reshaping(xtrain)
  # Define the hyperparameter grid to search
  naive_bayes_param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],
    'class_prior': [None, [0.3, 0.7]],
    'fit_prior': [True, False],
    'force_alpha': [True]  # Typically kept False
}

  # Create GridSearchCV object
  naive_bayes = GridSearchCV(naive_bayes_model, naive_bayes_param_grid, cv=5, scoring='accuracy')

  # Fit the grid search to your data
  naive_bayes.fit(xtrain, ytrain)

  # Get the best hyperparameters and model
  best_model = naive_bayes.best_estimator_
  best_params = naive_bayes.best_params_
  print("naive bayes Best Hyperparameters:", best_params)
  print("naive bayes Best Model:", best_model)
  return naive_bayes

In [311]:
print("-------------MultinomialNB_model------------")
multinomialNB_model = MultinomialNB()
MultinomialNB = multinominalNB_param_grid_search(multinomialNB_model, xtrain, ytrain)

-------------MultinomialNB_model------------
train data shape :  (8000, 784)
naive bayes Best Hyperparameters: {'alpha': 2.0, 'class_prior': None, 'fit_prior': False, 'force_alpha': True}
naive bayes Best Model: MultinomialNB(alpha=2.0, fit_prior=False)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py", line 765, in fit
    self._update_class_log_prior(class_prior=class_prior)
  File "/usr/local/lib/python3

In [312]:
joblib.dump(naive_bayes, 'MultinomialNB.pkl')

['MultinomialNB.pkl']

In [313]:
MultinomialNB = joblib.load('MultinomialNB.pkl')

In [314]:
accuracy,class_report,conf_matrix = model_evaluation(MultinomialNB,xtest,ytest)

train data shape :  (2000, 784)
predection classes [5 0 4 9 1 7 8 9 9 3]
true classes [5 0 8 9 6 7 7 7 7 9]
Accuracy: 52.20%
 Accuracy: 52.20%
 f1_score: 52.24%
 precision_score: 55.19%
 recall_score: 52.20%
confusion_matrix: 
 [[117  17   6   8  26   8   8   3   0   9]
 [  3 161   0  12   5   1   5   8   5   2]
 [ 10  45  83  25   7   1   3   3  10   1]
 [ 14  21  13 121   6   1   2   7   6   6]
 [ 33  30   7   6  81   1   4   8  36   6]
 [ 27  22   9   5   7  87  20   2  13   5]
 [ 18  27   0   1  15  18 123   4   5   0]
 [  9  30   9  13  19   1   1  99  15  18]
 [  9  35   8  13  19   8   7   5  91  10]
 [ 10  24   3  15  10   0   1  14  14  81]]


In [321]:
def MLP(X_train,y_train,):
  X_train = data_reshaping(X_train)
  mlp_classifier = MLPClassifier(hidden_layer_sizes=(8, 128,64,32), activation='relu', solver='adam',
                                       max_iter=1000, random_state=42,alpha=0.007,
                                       validation_fraction=0.3,verbose=1)
  mlp_classifier.fit(X_train, y_train)
  return mlp_classifier

In [322]:
print("-------------mlp_model------------")
mlp_model = MLP(xtrain, ytrain)


-------------mlp_model------------
train data shape :  (8000, 784)
Iteration 1, loss = 2.30169393
Iteration 2, loss = 2.18149996
Iteration 3, loss = 1.94984817
Iteration 4, loss = 1.74869025
Iteration 5, loss = 1.64575774
Iteration 6, loss = 1.53043021
Iteration 7, loss = 1.48930368
Iteration 8, loss = 1.43731194
Iteration 9, loss = 1.41879499
Iteration 10, loss = 1.39508744
Iteration 11, loss = 1.39064772
Iteration 12, loss = 1.37821251
Iteration 13, loss = 1.34792715
Iteration 14, loss = 1.36381150
Iteration 15, loss = 1.34444921
Iteration 16, loss = 1.33866023
Iteration 17, loss = 1.33185057
Iteration 18, loss = 1.31866628
Iteration 19, loss = 1.32122658
Iteration 20, loss = 1.29704723
Iteration 21, loss = 1.29872330
Iteration 22, loss = 1.28482175
Iteration 23, loss = 1.27532621
Iteration 24, loss = 1.28519751
Iteration 25, loss = 1.27075827
Iteration 26, loss = 1.27091527
Iteration 27, loss = 1.26822942
Iteration 28, loss = 1.27981857
Iteration 29, loss = 1.24892370
Iteration 30, 

In [323]:
joblib.dump(mlp_model, 'mlp_model.pkl')

['mlp_model.pkl']

In [324]:

mlp_model = joblib.load('mlp_model.pkl')
accuracy,class_report,conf_matrix = model_evaluation(mlp_model,xtest,ytest)

train data shape :  (2000, 784)
predection classes [5 0 1 9 6 7 9 9 7 9]
true classes [5 0 8 9 6 7 7 7 7 9]
Accuracy: 66.50%
 Accuracy: 66.50%
 f1_score: 66.02%
 precision_score: 66.42%
 recall_score: 66.50%
confusion_matrix: 
 [[160   1   7   8   4   3   2   2   5  10]
 [  4 170   3   0   3   1   3   7  10   1]
 [  1   6 138  18   3   4   4   7   3   4]
 [  6   5  10 147   5   1   0   3   5  15]
 [  9  17   3   1 146   1   5  10  10  10]
 [ 16   2  11   3   2 112  20   2  23   6]
 [ 16   5   5   2   7  12 156   0   8   0]
 [  5  18  11   8  18   0   0 115   5  34]
 [  7  27   4  14  29  12  11  14  82   5]
 [  8   2   4  16   8   1   0  29   0 104]]


In [ ]:
!pip install joblib==1.3.2

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [326]:
import gradio as gr

cnn_model = tf.keras.models.load_model("cnn_model.keras")

def predict_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = img.reshape(1,28, 28, 1)
    img = img / 255.0

    prediction = cnn_model.predict(img)
    predicted_class = np.argmax(prediction)
    return f"Predicted Class: {predicted_class}"

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="filepath"),
    outputs="text",
    title="Image Classification",
    description="Upload an image to get its predicted class."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fde06e82aa002e05cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [327]:
!cp -r /content/project/ /content/drive/MyDrive/Colab\ final/